### Project: 'MODELO PREDICTIVO DE CLASIFICACIÓN DE CHURN BASADO EN PATRONES DE CONTACTO PARA UNA STARTUP BOLIVIANA'  
#### Author: Flavia Davila Perez
#### Arquitecture: Medallion
##### Stage: Bronce (Extract)
Description: Extracted all metadata and data based on filters from the Pipedrive API, mapping and column selection.


### Libraries 

In [1]:
import os
import json
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv

In [2]:
import sys

sys.path.append('../scripts')
from pipedrive_api import *

### Credentials

In [3]:
load_dotenv()

# credentials
api_token_exp = os.getenv("api_token_exp")
api_token_com = os.getenv("api_token_com")
company_domain = os.getenv("company_domain")

### GET DATA

#### GET Metadata

In [4]:
# getting metadata - Deals
json_deals_metadata = deals_metadata(
    api_token_exp, 
    company_domain
    )
df_deals_metadata = pd.json_normalize(json_deals_metadata)

In [5]:
# getting metadata - Activities
json_act_metadata = activities_metadata(
    api_token_exp, 
    company_domain
    )
df_act_metadata = pd.json_normalize(json_act_metadata)

### GET Data from Filters

#### Activities

In [6]:
# getting activities from filter 13666
json_activities_com = get_activities(
    user_id = 0,
    filter_id=13666,
    done=True, 
    api_token=api_token_com,
    company_domain= company_domain  
)

In [7]:
# getting activities from filter
json_activities_exp = get_activities(
    user_id = 0,
    filter_id=13671,
    done=True, 
    api_token=api_token_exp,
    company_domain= company_domain  
)

#### Deals

In [8]:
# getting deals from filter All 'Mensual' Deals without 
json_deals_com = get_deals(
    filter_id=14375,
    api_token=api_token_com,
    company_domain= company_domain  
)

In [9]:
# getting deals from filter - EXP
json_deals_exp = get_deals(
    filter_id=14376,
    api_token=api_token_exp,
    company_domain= company_domain  
)

### GET fields

In [10]:
# getting activities fields
act_col = get_activity_fields(api_token_exp)

In [11]:
# getting deals fields
deals_col = get_deals_fields(api_token_exp)

### GET Stages

In [12]:
pipelines = get_pipeline_id(api_token_com)

Pipeline ID: 32, Name: 🟡 [ALL-001] Demo created
Pipeline ID: 36, Name: 🔴 [ALL-101] Leads Tibios
Pipeline ID: 22, Name: 🔴 [ALL-102] Leads Frios
Pipeline ID: 27, Name: 🔴 [ALL-103] Leads para Partners
Pipeline ID: 37, Name: 🟠 [BO-201] Inbound 
Pipeline ID: 1, Name: 🟠 [BO-202] Leads Calientes
Pipeline ID: 10, Name: 🟠 [PE-201] Inbound
Pipeline ID: 41, Name: 🟠 [PE-202] Leads Calientes
Pipeline ID: 8, Name: 🟠 [MX-201] Inbound
Pipeline ID: 42, Name: 🟠 [MX-202] Leads Calientes
Pipeline ID: 31, Name: 🟠 [ALL-201] Embajadores
Pipeline ID: 3, Name: 🟣 [BO-301] Experiencia
Pipeline ID: 16, Name: 🟣 [BO-302] Recomendados Experiencia
Pipeline ID: 38, Name: 🟣 [BO-303] Plan "Crece Mype"
Pipeline ID: 30, Name: 🟣 [PE-301] Experiencia
Pipeline ID: 9, Name: 🟣 [MX-301] Experiencia
Pipeline ID: 17, Name: 🟣 [ALL-301] Churn
Pipeline ID: 12, Name: 🟣 [ALL-302] Cuentas Consultores tuGerente
Pipeline ID: 43, Name: 🟣 [ECU-301] Experiencia
Pipeline ID: 45, Name: Embudo de prueba
Pipeline ID: 46, Name: Embudo Inbound LA

In [13]:
stages1 = get_stage_info(1, api_token_com)
stages3 = get_stage_info(3, api_token_com)
stages17 = get_stage_info(17, api_token_exp)

In [14]:
stages_1 = extract_stage_ids_and_names(stages1)
stages_3 = extract_stage_ids_and_names(stages3)
stages_17 = extract_stage_ids_and_names(stages17)

In [15]:
stages = stages_1 | stages_3 |stages_17

### JSON to DataFrame

#### Comercial

In [16]:
# json_activities to dataframe
df_activities_com = pd.json_normalize(json_activities_com)

# json_deals_com to dataframe
df_deals_com = pd.json_normalize(json_deals_com)

df_deals_com = df_deals_com.dropna(axis=1, how='all')

#### Experience

In [17]:
# json_activities to dataframe
df_activities_exp = pd.json_normalize(json_activities_exp)

# json_deals to dataframe
df_deals_exp = pd.json_normalize(json_deals_exp)

### Export Bronce Data

In [18]:
# Export Bronce Data to CSV

# Activities
df_activities_com.to_csv(r'..\data\output_bronce\activities_com.csv', index=False)
df_activities_exp.to_csv(r'..\data\output_bronce\activities_exp.csv', index=False)

# Deals
df_deals_com.to_csv(r'..\data\output_bronce\deals_com.csv', index=False)
df_deals_exp.to_csv(r'..\data\output_bronce\deals_exp.csv', index=False)


In [19]:
# Export Bronce Data to JSON

# JSON
with open(r'..\data\output_bronce\json_deals.json', "w") as json_file:
    json.dump(json_deals_metadata, json_file, indent=4)

with open(r'..\data\output_bronce\json_activities.json', "w") as json_file:
    json.dump(json_act_metadata, json_file, indent=4)

with open(r'..\data\output_bronce\act_col.json', "w") as json_file:
    json.dump(act_col, json_file, indent=4) 
    
with open(r'..\data\output_bronce\deals_col.json', "w") as json_file:
    json.dump(deals_col, json_file, indent=4) 

with open('stages_dict.json', 'w') as f:
    json.dump(stages, f, indent=4)